In [29]:
# multivariate multi-step stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Layer
from numpy import array
from numpy import hstack
from keras.layers import *
from keras.models import *
from keras import backend as K
import numpy as np
 
from sklearn.preprocessing import PowerTransformer
import pandas as pd

2

In [43]:

power = PowerTransformer(method='yeo-johnson')
 
# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(attention,self).__init__()
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context
    

def stable_sigmoid(x):

    sig = np.where(x < 0, np.exp(x)/(1 + np.exp(x)), 1/(1 + np.exp(-x)))
    return sig

def inverse_sigmoid(x):
    inv_sig = -np.log((1 / (x + 1e-8)) - 1)
    return inv_sig
    
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, 0:2], sequences[end_ix-1:out_end_ix, 1][1:3]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
     
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])

in_seq = np.transpose((in_seq1, in_seq2))
#np.array(pd.concat([pd.DataFrame(in_seq1),pd.DataFrame(in_seq2)],axis=1))

in_seq1 = in_seq[:,0]
in_seq2 = in_seq[:,1]

out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)

print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


n_features = X.shape[2]
# define model
model = Sequential()

"""
model.add(LSTM(100, activation='tanh', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='tanh'))
model.add(Dense(n_steps_out))
"""
model.add(Bidirectional(LSTM(128, activation='relu', return_sequences=True), input_shape=(n_steps_in, n_features)))
model.add((LSTM(128, activation='relu', return_sequences=True)))
#model.add(BatchNormalization())
model.add(attention(return_sequences=True)) #
model.add(Dropout(0.05))
model.add(Dense(n_steps_out))

model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
#x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = in_seq[-n_steps_in:]
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=1)
print(yhat)


(5, 3, 2) (5, 2)
[[10 15]
 [20 25]
 [30 35]] [45 55]
[[20 25]
 [30 35]
 [40 45]] [55 65]
[[30 35]
 [40 45]
 [50 55]] [65 75]
[[40 45]
 [50 55]
 [60 65]] [75 85]
[[50 55]
 [60 65]
 [70 75]] [85 95]
1/1 [==============================] - 0s 481ms/step
[[107.06367 117.60981]]


array([[[70, 75],
        [80, 85],
        [90, 95]]])

array([[70, 75],
       [80, 85],
       [90, 95]])